<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">

<h1 style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
M2.2 – العمل مع بيانات المناخ الشبكية
</h1>

<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
جزء من:
<a href="https://github.com/OpenClimateScience/M2-Computational-Climate-Science"><strong>علوم المناخ الحاسوبية</strong></a>
&nbsp;|&nbsp;
<strong>الدرس السابق</strong>
&nbsp;|&nbsp;
<strong>الدرس التالي</strong>
</p>

</div>


<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">

<strong>المحتويات:</strong>

<ul style="list-style-position: inside;">
  <li><a href="#Conceptual-and-computational-constraints">القيود المفاهيمية والحاسوبية</a></li>
  <li><a href="#A-terrestrial-water-storage-(TWS)-time-series">سلسلة زمنية لتخزين المياه الأرضية (<span dir="ltr">TWS</span>)</a>
    <ul>
      <li><a href="#Accessing-xarray-variables">الوصول إلى متغيرات <span dir="ltr">xarray</span></a></li>
    </ul>
  </li>
  <li><a href="#Thinking-about-multi-dimensional-arrays">التفكير في المصفوفات متعددة الأبعاد</a>
    <ul>
      <li><a href="#Updating-coordinate-systems">تحديث أنظمة الإحداثيات</a></li>
    </ul>
  </li>
  <li><a href="#Indexing-a-multi-dimensional-array">فهرسة مصفوفة متعددة الأبعاد</a></li>
  <li><a href="#Aggregating-along-the-axis-of-a-multi-dimensional-array">التجميع على طول محور في مصفوفة متعددة الأبعاد</a></li>
  <li><a href="#Resampling-a-time-series">إعادة أخذ العينات لسلسلة زمنية</a>
    <ul>
      <li><a href="#Handling-missing-data">التعامل مع البيانات المفقودة</a></li>
    </ul>
  </li>
  <li><a href="#Visualizing-variation-over-space">تصوير التباين عبر المكان</a></li>
  <li><a href="#Summary">الخلاصة</a></li>
</ul>

</div>


<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">

<h2 id="Conceptual-and-computational-constraints" style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
القيود المفاهيمية والحاسوبية
</h2>

<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
مع تحسّن تقنيات الحوسبة وتزايد توفر مجموعات بيانات الأقمار الصناعية، يزداد حجم بيانات المناخ وتعقيدها وتواترها بسرعة. وقد يؤدي ذلك إلى مشكلتين للباحثين ومديري الموارد الذين يسعون لاستخدام بيانات المناخ. أولًا، قد يكون من الصعب تصوّر حجم ومقياس بعض مجموعات بيانات المناخ، مما قد يسبب تحديات في إدارة البيانات والمشاريع (Jain وآخرون، 2022). ثانيًا، قد يكون من الصعب معالجة وتحليل مجموعات بيانات مناخية تكون أحيانًا كبيرة جدًا بحيث لا تتسع في ذاكرة الحاسوب، أو معقّدة إلى حد يجعل الحسابات غير عملية.
</p>

<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
في هذا الدرس، سنستكشف كيفية إدارة بعض هذه القضايا. وبما أننا نتعلّم، سنستخدم مجموعة بيانات ليست كبيرة أو معقّدة مثل غيرها مما قد نصادفه لاحقًا، لكنها ستكون مثالًا جيدًا على مجموعات أصعب وكيفية التعامل معها.
</p>

</div>


In [ ]:
import xarray as xr
import earthaccess
import numpy as np
from matplotlib import pyplot

auth = earthaccess.login()

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">

<h2 id="A-terrestrial-water-storage-(TWS)-time-series" style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
سلسلة زمنية لتخزين المياه الأرضية (<span dir="ltr">TWS</span>)
</h2>

<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
لاكتساب خبرة أكبر في العمل مع بيانات المناخ الشبكية، وخاصةً السلاسل الزمنية لبيانات مناخية شبكية، سنستخدم مجموعة بيانات لتخزين المياه الأرضية (<span dir="ltr">TWS</span>) تم إنتاجها اعتمادًا على بيانات مهمتين قمريتين: مهمة <span dir="ltr">GRACE</span> وخليفتها <span dir="ltr">GRACE-FO</span>.
</p>

<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
يشمل <span dir="ltr">TWS</span> كل المياه على سطح اليابسة وتحتها، بما في ذلك الغطاء الثلجي، والجداول والخزانات السطحية، والمياه الجوفية. وقد تم بالفعل تحويل
<a href="https://podaac.jpl.nasa.gov/dataset/TELLUS_GRAC-GRFO_MASCON_CRI_GRID_RL06.1_V3">مجموعة بيانات <span dir="ltr">TWS</span> التي سنعمل عليها من مهمتي GRACE وGRACE-FO</a>
إلى <em>شواذ شهرية</em>.
</p>

<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
&#x1F449; <a href="https://grace.jpl.nasa.gov/about/faq/"><strong>اقرأ المزيد عن بيانات GRACE/GRACE-FO هنا.</strong></a>
</p>

<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
تذكّر أن <em>الشواذ</em> هي إحدى طرق تمثيل التباين حول متوسط طويل الأمد. فهي تخبرنا كيف تُقارن قيمة معيّنة (يوم/شهر/سنة) بالسياق عبر الزمن؛ مثلًا: هل هذه سنة جافة بشكل استثنائي؟
</p>

</div>


<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">

<h4 style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
&#x1F3AF; أفضل ممارسة
</h4>

<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
<em>سنقوم الآن بتنزيل بعض البيانات الخام! أنت تعرف ما يجب فعله.</em>
أنشئ مجلدًا جديدًا باسم <span dir="ltr"><code>GRACE</code></span> داخل المجلد <span dir="ltr"><code>data_raw</code></span>.
</p>

</div>


In [ ]:
results = earthaccess.search_data(short_name = 'TELLUS_GRAC-GRFO_MASCON_CRI_GRID_RL06.1_V3')
results

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">
<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">تم إعداد مجموعة البيانات هذه على شكل ملف واحد بصيغة <span dir="ltr">netCDF</span>.</p>
</div>


In [ ]:
earthaccess.download(results, 'data_raw/GRACE')

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">

<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
<strong>قد تظهر لك رسالة خطأ عند محاولة تنزيل هذه البيانات بسبب مشكلة في خوادم ناسا.</strong>
إذا حدث ذلك، لاحظ أن هناك رابطًا مباشرًا واحدًا لهذه المجموعة (مُدرج ضمن المخرجات أعلاه)، ويمكنك استخدامه لتنزيل البيانات مباشرةً. ينبغي أن يكون:
</p>

<ul style="list-style-position: inside;">
<li>
<a dir="ltr" href="https://archive.podaac.earthdata.nasa.gov/podaac-ops-cumulus-protected/TELLUS_GRAC-GRFO_MASCON_CRI_GRID_RL06.1_V3/GRCTellus.JPL.200204_202311.GLO.RL06.1M.MSCNv03CRI.nc">https://archive.podaac.earthdata.nasa.gov/podaac-ops-cumulus-protected/TELLUS_GRAC-GRFO_MASCON_CRI_GRID_RL06.1_V3/GRCTellus.JPL.200204_202311.GLO.RL06.1M.MSCNv03CRI.nc</a>
(اضغط للتنزيل)
</li>
</ul>

</div>


In [ ]:
ds = xr.open_dataset('data_raw/GRACE/GRCTellus.JPL.200204_202311.GLO.RL06.1M.MSCNv03CRI.nc')
ds

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">
<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
تمثل بيانات <span dir="ltr">GRACE/GRACE-FO</span> بالفعل <em>شواذ</em>، أي تغيّرًا في إجمالي تخزين المياه مقارنةً بفترة زمنية معيّنة (أو «حقبة»). ما هي هذه الفترة؟ يمكننا معرفة ذلك من خلال <strong>سمات الملف</strong> (على مستوى الملف) داخل ملف <span dir="ltr">netCDF</span>.
</p>
</div>


In [ ]:
ds.attrs['time_mean_removed']

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">

<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
يمكننا أيضًا التحقق من ذلك عبر النظر إلى سلسلة زمنية لمتوسط الشواذ الشهرية العالمية لتخزين المياه الأرضية (<span dir="ltr">TWS</span>). خلال الفترة 2004–2009 تبدو الشواذ متمركزة حول الصفر، وهو ما نتوقعه لأن متوسط تلك الفترة قد طُرح من البيانات.
</p>

<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
كما نلاحظ اتجاهًا هبوطيًا طويل الأمد. ما الذي قد يفسّر ذلك؟ يُرجَّح أن يكون الفاقد العالمي من الجليد الأرضي هو العامل الرئيسي. يقدّر التقرير التقييمي السادس (<span dir="ltr">AR6</span>) للهيئة الحكومية الدولية المعنية بتغير المناخ (<span dir="ltr">IPCC</span>) أن ذوبان الأنهار الجليدية والصفائح الجليدية بين 2006 و2018 ساهم بحوالي 2 سم من ارتفاع مستوى سطح البحر (IPCC 2021، الفصل 9)، وهو ما يتسق مع ما نراه هنا (والمحور الرأسي في الرسم أيضًا بالسنتيمتر). كما يلعب احترار الغلاف الجوي دورًا كذلك، لأن الهواء الأدفأ يستطيع حمل كمية أكبر من بخار الماء.
</p>

</div>


In [ ]:
ds['lwe_thickness'].mean(['lon','lat']).plot()

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">

<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
يعود الشذوذ في السلسلة الزمنية حول 2017–2018 إلى أن مهمة <span dir="ltr">GRACE</span> انتهت في أكتوبر 2017، ولم يُطلق القمر <span dir="ltr">GRACE-FO</span> حتى مايو 2018؛ لذلك لا توجد قيم بيانات لتلك الأشهر.
</p>

<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
ويمكننا النظر إلى أكثر من سلسلة زمنية: فهذه بيانات مكانية أيضًا. لنلقِ نظرة على شواذ <span dir="ltr">TWS</span> العالمية لأول تاريخ.
</p>

</div>


In [ ]:
ds['lwe_thickness'][0].plot(vmin = -200, vmax = 200, cmap = 'RdBu')

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">
<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
في الرسم أعلاه، الذي يُظهر الشذوذ الشهري لتخزين المياه الأرضية في أبريل 2002، تشير الألوان الباردة إلى شذوذ إيجابي (أي مياه أكثر من المعتاد في أبريل)، وتشير الألوان الدافئة إلى شذوذ سلبي (أي مياه أقل من المعتاد في أبريل).
</p>
</div>


<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">

<h3 id="Accessing-xarray-variables" style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
الوصول إلى متغيرات <span dir="ltr">xarray</span>
</h3>

<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
لاحظ أن كلًا من الإحداثيات والمتغيرات داخل كائنات <span dir="ltr">xarray.Dataset</span> يمكن الوصول إليها بطريقتين مختلفتين:
</p>

<ul style="list-style-position: inside;">
<li>باستخدام اسم الإحداثي أو المتغير كمفتاح في قاموس: <span dir="ltr"><code>ds['lwe_thickness']</code></span> أو <span dir="ltr"><code>ds['time']</code></span></li>
<li>أو باستخدام الاسم كسمة: <span dir="ltr"><code>ds.lwe_thickness</code></span> أو <span dir="ltr"><code>ds.time</code></span></li>
</ul>

<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
وبغضّ النظر عن الصيغة التي تستخدمها، فالنتيجة واحدة.
</p>

</div>


<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">

<hr />

<h2 id="Thinking-about-multi-dimensional-arrays" style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
التفكير في المصفوفات متعددة الأبعاد
</h2>

<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
<strong>لنعُد لفتح مجموعة البيانات باستخدام <span dir="ltr">open_mfdataset()</span>.</strong>
تذكّر أننا نستخدم <span dir="ltr"><code>open_mfdataset()</code></span> من <span dir="ltr">xarray</span> عندما نريد فتح عدة ملفات كما لو كانت مجموعة بيانات واحدة.
هنا لدينا ملف واحد فقط، لكننا سنعاملُه كما لو كان عدة ملفات لأنه يحتوي على صورة عالمية لشذوذ <span dir="ltr">TWS</span> لعدة تواريخ.
</p>

</div>


In [ ]:
ds = xr.open_mfdataset('data_raw/GRACE/GRCTellus.JPL.200204_202311.GLO.RL06.1M.MSCNv03CRI.nc')

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">
<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">كما رأينا، يمكن تمثيل ملفات <span dir="ltr">netCDF4</span> و<span dir="ltr">HDF5</span> على شكل كائنات <span dir="ltr">xarray.Dataset</span>.</p>
</div>


In [ ]:
type(ds)

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">

<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
يمكن أن تحتوي <span dir="ltr">Dataset</span> على أكثر من متغير واحد، وكل متغير يُمثَّل كمصفوفة متعددة الأبعاد.
</p>

<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
هناك طرق مختلفة لتمثيل المصفوفات في بايثون. عند استخدام <span dir="ltr">xarray</span>، تُعرض المتغيرات ككائنات <span dir="ltr">DataArray</span>.
</p>

</div>


In [ ]:
type(ds['lwe_thickness'])

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">
<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
عادةً ما يكون <span dir="ltr">xarray.DataArray</span> نوعًا خاصًا من مصفوفات <span dir="ltr">NumPy</span>. لكن عند استخدام <span dir="ltr"><code>xarray.open_mfdataset()</code></span> نحصل على نوع جديد من المصفوفات.
</p>
</div>


In [ ]:
type(ds['lwe_thickness'].data)

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">

<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
<strong>تُستخدم مكتبة <span dir="ltr">dask</span> تلقائيًا لتمثيل المصفوفة متعددة الأبعاد الأساسية.</strong>
وذلك لأن استخدام <span dir="ltr"><code>open_mfdataset()</code></span> يوحي بأننا قد نتعامل مع مصفوفة كبيرة، لأننا نفتح عدة ملفات.
</p>

<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
<strong>وباستعمال <span dir="ltr">xarray.open_mfdataset()</span> لتكديس ملفات متعددة في سلسلة زمنية، اكتسبنا أيضًا طريقة جديدة للتفكير في بياناتنا.</strong>
كان كل ملف منفرد يمثّل عمليًا صورة ثنائية الأبعاد: شذوذ <span dir="ltr">TWS</span> على شبكة خط العرض–خط الطول. وعند تكديس تواريخ متعددة فوق بعضها، نحصل على <strong>مكعّب بيانات</strong> ثلاثي الأبعاد بمحاور X (خط الطول) وY (خط العرض) ومحور الزمن. ويمكن لكائنات <span dir="ltr">xarray.Dataset</span> إظهار رسم توضيحي مفيد داخل <span dir="ltr">Jupyter Notebook</span>.
</p>

</div>
